# Seguridad a nivel de renglón

Paso 1. Crear base de datos de prueba

In [1]:
CREATE DATABASE [TestDB_RLS] CONTAINMENT = PARTIAL
GO
USE TestDB_RLS
GO
CREATE SCHEMA demo
GO
CREATE TABLE demo.customers (
  id int not null,
  name nvarchar(100) not null,
  country char(2) not null,
  primary key (id)
)
INSERT INTO demo.customers VALUES
(1, 'Andrea', 'AR'),
(2, 'Bruno', 'BE'),
(3, 'Charles', 'CA'),
(4, 'Daniella', 'DK'),
(5, 'Eric', 'ES')

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(5 rows affected)

Total execution time: 00:00:00.735

Paso 2. Crear usuarios de prueba

In [2]:
CREATE USER EU_testuser WITH PASSWORD = 'Galleta_09';
CREATE USER NA_testuser WITH PASSWORD = 'Galleta_09';

Commands completed successfully.

Total execution time: 00:00:00.011

In [3]:
GRANT SELECT ON demo.customers to EU_testuser;
GRANT SELECT ON demo.customers to NA_testuser;

Commands completed successfully.

Total execution time: 00:00:00.003

Paso 3. Crear función de filtrado

In [4]:
CREATE FUNCTION demo.RestrictCustomers(@country as sysname)
RETURNS TABLE WITH SCHEMABINDING AS
RETURN SELECT 1 AS res
WHERE
(@country IN ('DK', 'BE', 'ES') AND user_name() LIKE 'EU_%') OR
(@country IN ('US', 'CA') AND user_name() LIKE 'NA_%') OR
(suser_name() = 'sa')

Commands completed successfully.

Total execution time: 00:00:00.005

Paso 4. Aplicar la función de filtrado como política de seguridad

In [5]:
CREATE SECURITY POLICY CustomersPolicy
ADD FILTER PREDICATE demo.RestrictCustomers(country)
ON demo.customers
WITH (state = on)

Commands completed successfully.

Total execution time: 00:00:00.004

Paso 5. Ejecutar el comando como EU\_testuser y el usuario NA\_testuser

In [6]:
EXECUTE AS user = 'EU_testuser'
select * from demo.customers
REVERT

(3 rows affected)

Total execution time: 00:00:00.013

id,name,country
2,Bruno,BE
4,Daniella,DK
5,Eric,ES


In [13]:
EXECUTE AS user = 'NA_testuser'
select * from demo.customers
REVERT

(1 row affected)

Total execution time: 00:00:00.010

id,name,country
3,Charles,CA


## Aplicar seguridad de renglón para operaciones de escritura

In [7]:
ALTER SECURITY POLICY CustomersPolicy
ADD BLOCK PREDICATE demo.RestrictCustomers(country)
ON demo.customers AFTER INSERT

Commands completed successfully.

Total execution time: 00:00:00.005

In [8]:
GRANT INSERT ON SCHEMA::demo TO EU_testuser

Commands completed successfully.

Total execution time: 00:00:00.001

Pruebas de inserción

In [9]:
EXECUTE AS user = 'EU_testuser'
INSERT INTO demo.customers VALUES (100, 'Didi', 'DK')
REVERT

(1 row affected)

Total execution time: 00:00:00.008

In [10]:
EXECUTE AS user = 'EU_testuser'
INSERT INTO demo.customers VALUES (150, 'Argento', 'AR')
REVERT

: Msg 33504, Level 16, State 1, Line 2
The attempted operation failed because the target object 'TestDB_RLS.demo.customers' has a block predicate that conflicts with this operation. If the operation is performed on a view, the block predicate might be enforced on the underlying table. Modify the operation to target only the rows that are allowed by the block predicate.

The statement has been terminated.

Total execution time: 00:00:00.008